In [ ]:
from fastai.vision import *
import pandas as pd

In [ ]:
df = pd.read_csv('../input/hp-2020/jh_2020/train.csv')
test_df = pd.read_csv('../input/hp-2020/jh_2020/test.csv')
df.head()


In [ ]:
tfms = get_transforms(do_flip = True,flip_vert= True, max_rotate= 50, max_lighting= 0.1, max_warp = 0)

In [ ]:
data = ImageDataBunch.from_df('../input/hp-2020/jh_2020/images', df,ds_tfms=tfms,label_delim= None,valid_pct=0.2,fn_col=0, label_col=1 , size=299,bs=64).normalize(imagenet_stats)
data.path = pathlib.Path('.')
#data.normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,7))

In [ ]:

print(data.classes)

In [ ]:
learn =cnn_learner(data,models.resnet50,pretrained=True,metrics=[accuracy])


In [ ]:
learn.fit_one_cycle(4)


In [ ]:
#learn.save("fastai_model")
#learn = learn.load("fastai_model")

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)


In [ ]:
interp.plot_top_losses(9, figsize=(7,7))



In [ ]:
interp.plot_confusion_matrix(figsize=(8,8), dpi=60)

In [ ]:
learn.unfreeze()


In [ ]:
learn.fit_one_cycle(3)

In [ ]:
#learn.save("../input/output/fastai_model_unfreeze")
#learn = learn.load("fastai_model_unfreeze")

In [ ]:

# #print(os.listdir("../input/fastai-pretrained-models"))
# PATH = "../input/emergency-vehicle-detection"
# !mkdir -p /root/.cache/torch/checkpoints/
# !cp ../input/resnet50-19c8e357.pth /root/.cache/torch/checkpoints/resnet50-19c8e357.pth

#learn.model_dir('/root/.cache/torch/checkpoints/resnet50-19c8e357.pth')
learn.lr_find()

In [ ]:
learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(20,max_lr=slice(1e-4,1e-3))


In [ ]:
learn.freeze()
learn.save('car_classification1')

In [ ]:
path = '../input/hp-2020/jh_2020/images/'
emergency_or_not = []
for i in test_df['image_names']:
    img = open_image(path + i)
    pred_class,pred_idx, outputs = learn.predict(img)
    emergency_or_not.append(pred_class.obj)

In [ ]:
test_df['emergency_or_not'] = emergency_or_not

In [ ]:
test_df.to_csv('fastai1.csv', index = False)